# DSCI 100 - Proposal
### project-003-7: Nicole Campbell, Michelle Kim, Jason Liu, Zahra Raza

## Introduction
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Cras in porttitor magna. Vivamus a odio quis nisi consectetur posuere. Ut et dolor lectus. Aenean lobortis sollicitudin viverra. Vestibulum elementum hendrerit pulvinar. Cras massa lacus, condimentum et tincidunt sit amet, gravida vitae felis. Mauris sit amet eros eget mi malesuada gravida vestibulum quis nisl. Etiam a mollis turpis. Pellentesque eu elit at sapien dapibus tempus non ut quam. In hac habitasse platea dictumst. Pellentesque congue placerat odio non porttitor. Nulla dictum venenatis dui, in malesuada augue luctus quis. Sed bibendum orci ut sem pretium commodo sit amet quis dolor. Maecenas maximus vulputate diam, nec commodo sapien faucibus quis.

In [ ]:
### Run this cell before continuing.
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [ ]:
# This code was part of student-merge.R that came with the data
d1=read.table("student-mat.csv",sep=";",header=TRUE)
d2=read.table("student-por.csv",sep=";",header=TRUE)

d3=merge(d1,d2,by=c("school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"))
print(nrow(d3)) # 382 students